In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Merging Data

In [2]:
df_2018 = pd.read_csv("data/City_wise_raw_data_1Hr_2018_Asansol_1Hr.csv")
df_2019 = pd.read_csv("data/City_wise_raw_data_1Hr_2019_Asansol_1Hr.csv")
df_2020 = pd.read_csv("data/City_wise_raw_data_1Hr_2020_Asansol_1Hr.csv")
df_2021 = pd.read_csv("data/City_wise_raw_data_1Hr_2021_Asansol_1Hr.csv")
df_2022 = pd.read_csv("data/City_wise_raw_data_1Hr_2022_Asansol_1Hr.csv")
df_2023 = pd.read_csv("data/City_wise_raw_data_1Hr_2023_Asansol_1Hr.csv")
df_2024 = pd.read_csv("data/City_wise_raw_data_1Hr_2024_Asansol_1Hr.csv")

<b>Null Value Percentage of each year's dataset</b>
- 2018: 23%
- 2019 : 6%
- 2020 : 17%
- 2021 : 11%
- 2022 : 5%
- 2023 : 9%
- 2024 : 0.8%

According to cpcb data below 30% null values can be used for forecasting 

In [3]:
df_concat = pd.concat([df_2018, df_2019, df_2020, df_2021, df_2022, df_2023, df_2024], ignore_index=True)

In [4]:
print(df_concat.columns)

Index(['Timestamp', 'PM2.5 (µg/m³)', 'PM10 (µg/m³)', 'NO (µg/m³)',
       'NO2 (µg/m³)', 'NOx (ppb)', 'NH3 (µg/m³)', 'SO2 (µg/m³)', 'CO (mg/m³)',
       'Ozone (µg/m³)', 'Benzene (µg/m³)', 'Toluene (µg/m³)', 'Xylene (µg/m³)',
       'O Xylene (µg/m³)', 'Eth-Benzene (µg/m³)', 'MP-Xylene (µg/m³)',
       'AT (°C)', 'RH (%)', 'WS (m/s)', 'WD (deg)', 'RF (mm)', 'TOT-RF (mm)',
       'SR (W/mt2)', 'BP (mmHg)', 'VWS (m/s)'],
      dtype='object')


<b>We will be using 4 pollutants : PM2.5, PM10, NOx, SO2</b>

In [5]:
df_merged = df_concat[["Timestamp", "PM2.5 (µg/m³)", "PM10 (µg/m³)", "NOx (ppb)", "SO2 (µg/m³)"]]

In [6]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61368 entries, 0 to 61367
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Timestamp      61368 non-null  object 
 1   PM2.5 (µg/m³)  55570 non-null  float64
 2   PM10 (µg/m³)   55892 non-null  float64
 3   NOx (ppb)      54987 non-null  float64
 4   SO2 (µg/m³)    55725 non-null  float64
dtypes: float64(4), object(1)
memory usage: 2.3+ MB


In [7]:
df_merged.head(2)

,Timestamp,PM2.5 (µg/m³),PM10 (µg/m³),NOx (ppb),SO2 (µg/m³)
0,2018-01-01 00:00:00,NaN,NaN,NaN,NaN
1,2018-01-01 01:00:00,NaN,NaN,NaN,NaN


In [8]:
df_merged.tail(2)

,Timestamp,PM2.5 (µg/m³),PM10 (µg/m³),NOx (ppb),SO2 (µg/m³)
61366,2024-12-31 22:00:00,61.91,128.83,14.76,7.04
61367,2024-12-31 23:00:00,60.65,118.69,14.97,6.62


<b>Unit of Pollutants</b>  

NOx : ppb - parts per billion  
CO : mg/m³ - milligram per cubic meter  
Remaining 6 pollutants : µg/m³ - microgram per cubic meter  

1 mg = 1000 µg

In [9]:
df_merged.to_csv("data/Asansol Merged.csv")

# Basic Preprocessing

In [10]:
df = df_merged.copy()

In [11]:
df["Timestamp"] = pd.to_datetime(df["Timestamp"])
df = df.set_index("Timestamp")

df.duplicated().sum()

4571

All 4550 duplicates are rows with all columns as null values

In [12]:
#df = df.rename(columns = {"PM2.5 (µg/m³)":"PM2.5", "PM10 (µg/m³)":"PM10", "NO (µg/m³)":"NO", "NO2 (µg/m³)":"NO2", "NOx (ppb)":"NOx", "NH3 (µg/m³)":"NH3", "SO2 (µg/m³)":"SO2", "CO (mg/m³)":"CO"})

df = df.rename(columns = {"PM2.5 (µg/m³)":"PM2.5", "PM10 (µg/m³)":"PM10", "NOx (ppb)":"NOx", "SO2 (µg/m³)":"SO2"})

In [13]:
df.head(2)

,PM2.5,PM10,NOx,SO2
Timestamp,,,,
2018-01-01 00:00:00,NaN,NaN,NaN,NaN
2018-01-01 01:00:00,NaN,NaN,NaN,NaN


### Remove large blocks of Null Values

In [14]:
df = df[df.index >= "2018-02-01 10:00:00"]   # Remove entire january month due to null values

In [15]:
df.isnull().all(axis=1).sum()

3812

In [16]:
df["row_null"] = df.isnull().all(axis=1).astype(int)

In [17]:
df.head(2)

,PM2.5,PM10,NOx,SO2,row_null
Timestamp,,,,,
2018-02-01 10:00:00,NaN,NaN,24.72,NaN,0
2018-02-01 11:00:00,131.02,238.25,31.02,30.63,0


In [18]:
df.shape

(60614, 5)

In [19]:
'''block_center = (df['row_null']
                .rolling(window=15, center=True)
                .sum() == 15)


block_full = np.zeros(len(df), dtype=bool)

for idx in df[block_center].index:
    i = df.index.get_loc(idx)
    block_full[max(0, i-7):min(len(df), i+8)] = True

df['block_null'] = block_full.astype(int)

df.to_csv("C:/Users/91741/Desktop/df_block.csv")'''

'block_center = (df[\'row_null\']\n                .rolling(window=15, center=True)\n                .sum() == 15)\n\n\nblock_full = np.zeros(len(df), dtype=bool)\n\nfor idx in df[block_center].index:\n    i = df.index.get_loc(idx)\n    block_full[max(0, i-7):min(len(df), i+8)] = True\n\ndf[\'block_null\'] = block_full.astype(int)\n\ndf.to_csv("C:/Users/91741/Desktop/df_block.csv")'

In [20]:
df = df[df["row_null"] != 1]
df = df.drop(["row_null"], axis=1)

In [21]:
df.isnull().all(axis=1).sum()

0

In [22]:
df.isnull().sum()

PM2.5    1232
PM10      910
NOx      1815
SO2      1077
dtype: int64

In [23]:
# Step 1: Interpolate missing values (time-aware, continuous)
df.interpolate(method='time', limit=12, inplace=True)

# Step 2: Fill short edge gaps if any remain
df.fillna(method='ffill', limit=3, inplace=True)
df.fillna(method='bfill', limit=3, inplace=True)

C:\Users\91741\AppData\Local\Temp\ipykernel_12996\1354377464.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', limit=3, inplace=True)
C:\Users\91741\AppData\Local\Temp\ipykernel_12996\1354377464.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', limit=3, inplace=True)


In [24]:
df.dropna(inplace=True)

In [25]:
df.isnull().sum()

PM2.5    0
PM10     0
NOx      0
SO2      0
dtype: int64

In [30]:
df_train = df.loc[:'2023-12-31', ]
df_val = df.loc['2024-01-01':, ]

Split dataset into train-validation  
Train Set: 2018 to 2023  
Validation Set: 2024  
Test Set: 2025

In [44]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

train_sc = scaler.fit_transform(df_train)
val_sc = scaler.transform(df_val)

train_scaled = pd.DataFrame(train_sc, columns=scaler.get_feature_names_out(), index = df_train.index)
val_scaled = pd.DataFrame(val_sc, columns=scaler.get_feature_names_out(), index = df_val.index)

LSTM requires features in same scale

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 55882 entries, 2018-02-01 10:00:00 to 2024-12-31 23:00:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PM2.5   55882 non-null  float64
 1   PM10    55882 non-null  float64
 2   NOx     55882 non-null  float64
 3   SO2     55882 non-null  float64
dtypes: float64(4)
memory usage: 4.1 MB


# Modeling

### LSTM

In [27]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):

    def __init__(self, input_size, ouptut_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm(input_size=input_size, num_layers=num_layers, hidden_size=hidden_size)
        self.fc = nn.Linear(hidden_size, ouptut_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out
    
    def train_one_epoch(model, dataloader, optimizer, loss_fn, device):
        model.train()
        total_loss = 0

        for X_batch, y_batch in dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            
            output = model(X_batch)
            loss = loss_fn(output, y_batch)
            loss.backward()
            optimizer.step()
            total_loss = loss.item()

        return total_loss / len(dataloader)
    
    def evaluate(model, dataloader, loss_fn, device):
        model.eval()
        total_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in dataloader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                output = model(X_batch)
                loss = loss_fn(output, y_batch)
                total_loss += loss.item()
        return total_loss / len(dataloader)

In [28]:
# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel().to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training
num_epochs = 20
for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, loss_fn, device)
    val_loss = evaluate(model, val_loader, loss_fn, device)
    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")


TypeError: LSTMModel.__init__() missing 4 required positional arguments: 'input_size', 'ouptut_size', 'hidden_size', and 'num_layers'